# Imports

In [4]:
import numpy as np
import itertools
import datetime

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import os
from amuse.units import units

from amuse.lab import *
from amuse.io import write_set_to_file, read_set_from_file

#import libraries
from amuse.lab import *
from amuse.community.fi.interface import Fi
from amuse.datamodel import Particles

from itertools import combinations

# from run_6body_with_auto_hydro import run_6_body_simulation
# from gif_plotting import visualize_frames

/home/yang-taotao/miniforge3/envs/amuse-py313/lib/python3.13/site-packages/amuse/support/options.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [5]:
def visualize_frames(frames, run_label="test"):
    """
    Visualize AMUSE simulation frames and produce a GIF.
    - Color-coded by stellar mass (with colorbar)
    - Fixed system COM (no recentering after collision)
    """

    if len(frames) == 0:
        print("⚠️ No frames provided.")
        return

    os.makedirs("Gif-6body", exist_ok=True)

    # --- Mass colormap setup ---
    all_masses = np.array([p.mass.value_in(units.MSun) for f in frames for p in f])
    m_min, m_max = all_masses.min(), all_masses.max()
    cmap = plt.get_cmap("plasma")
    norm = mcolors.Normalize(vmin=m_min, vmax=m_max)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)

    def mass_to_color(mass):
        return cmap(norm(mass))

    # --- Figure setup ---
    fig, ax = plt.subplots(figsize=(8, 6))
    sc = ax.scatter([], [], s=[])
    time_text = ax.text(0.02, 0.95, "", transform=ax.transAxes, fontsize=12,
                        verticalalignment='top', color='black')

    ax.set_xlabel("x [AU]", fontsize=12)
    ax.set_ylabel("y [AU]", fontsize=12)

    # Colorbar
    cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("Mass [M$_\\odot$]", fontsize=12)
    cbar.ax.tick_params(labelsize=10)

    # --- Fixed reference: initial system COM ---
    frame0 = frames[0]
    com0_x = np.mean([p.x.value_in(units.AU) for p in frame0])
    com0_y = np.mean([p.y.value_in(units.AU) for p in frame0])

    # --- Initialization ---
    def init():
        sc.set_offsets(np.empty((0, 2)))
        ax.set_xlim(-2000, 2000)
        ax.set_ylim(-2000, 2000)
        return sc, time_text

    # --- Frame update ---
    def update(frame_index):
        frame = frames[frame_index]
        x = np.array([p.x.value_in(units.AU) for p in frame])
        y = np.array([p.y.value_in(units.AU) for p in frame])
        masses = np.array([p.mass.value_in(units.MSun) for p in frame])

        sizes = np.clip(masses * 2, 10, 500)
        colors = [mass_to_color(m) for m in masses]

        # Keep coordinates fixed relative to initial COM
        x_rel = x - com0_x
        y_rel = y - com0_y

        sc.set_offsets(np.c_[x_rel, y_rel])
        sc.set_sizes(sizes)
        sc.set_color(colors)

        # Static frame (no zoom, no recenter)
        ax.set_xlim(-1200, 1200)
        ax.set_ylim(-1200, 1200)

        dt = 5  # years per frame
        t = frame_index * dt
        time_text.set_text(f"t = {t:.0f} yr")

        return sc, time_text

    # --- Animation ---
    ani = FuncAnimation(fig, update, frames=len(frames),
                        init_func=init, interval=50, blit=False, repeat=False)

    gif_filename = os.path.join("Gif-6body", f"encounter_evolution_{run_label}.gif")
    writer = PillowWriter(fps=12)
    ani.save(gif_filename, writer=writer)

    print(f" GIF saved as {gif_filename}")
    plt.close(fig)

In [6]:
def pairwise_separations(particles):
    """Return list of (i,j, separation_length) for particle set."""
    pairs = []
    for i, j in combinations(range(len(particles)), 2):
        sep = (particles[i].position - particles[j].position).length()
        pairs.append((i, j, sep))
    return pairs

def detect_close_pair(particles, radii, buffer_factor=0.4):
    """
    Detects the first close pair based on radii overlap with a simple buffer.
    Returns (i, j, sep) or None.

    For some reason after testing an ideal buffer should be 0.3<b<0.4. 
    Idk why but you can test this yourself
    Higher values seem to give unphysical encounters.
    Smaller values might be possible
    """
    for i, j, sep in pairwise_separations(particles):
        # Skip invalid radii
        if not np.isfinite(radii[i].value_in(units.RSun)) or not np.isfinite(radii[j].value_in(units.RSun)):
            continue
        if radii[i] <= 0 | units.RSun or radii[j] <= 0 | units.RSun:
            continue

        # Use a simple buffer multiplier
        threshold = (radii[i] + radii[j]) * buffer_factor
        if sep < threshold:
            return (i, j, sep)
    return None



def create_sph_star(mass, radius, n_particles=1000, u_value=None, pos_unit=units.AU):
    """
    Create a uniform-density SPH star with safer defaults.
    mass, radius: AMUSE quantities
    pos_unit: coordinate unit for output positions
    """
    sph = Particles(n_particles)

    # set per-particle mass (AMUSE broadcasts quantity)
    sph.mass = (mass / n_particles)

    # sample radius uniformly in volume (keep units)
    # convert radius to meters for numpy sampling then reattach unit
    r_vals = (radius.value_in(units.m) * np.random.random(n_particles)**(1/3)) | units.m
    theta = np.arccos(2.0 * np.random.random(n_particles) - 1.0)
    phi = 2.0 * np.pi * np.random.random(n_particles)

    x = r_vals * np.sin(theta) * np.cos(phi)
    y = r_vals * np.sin(theta) * np.sin(phi)
    z = r_vals * np.cos(theta)

    # attach coordinates in requested unit
    sph.x = x.in_(pos_unit)
    sph.y = y.in_(pos_unit)
    sph.z = z.in_(pos_unit)

    # velocities zero in star frame
    sph.vx = 0. | units.kms
    sph.vy = 0. | units.kms
    sph.vz = 0. | units.kms

    # internal energy estimate
    if u_value is None:
        # virial-ish estimate: u ~ 0.2 * G M / R  (units J/kg)
        u_est = 0.2 * (constants.G * mass / radius)
        sph.u = u_est
    else:
        sph.u = u_value

    # compute a mean inter-particle spacing in meters and set h_smooth to a safe fraction
    mean_sep = ( (4/3.0)*np.pi*(radius.value_in(units.m)**3) / n_particles )**(1/3) | units.m
    # choose smoothing length ~ 1.2 * mean_sep (safe number of neighbors)
    sph.h_smooth = (1.2 * mean_sep).in_(pos_unit)

    return sph


def make_sph_from_two_stars(stars, n_sph_per_star=5000, u_value=None, pos_unit=units.AU):
    if len(stars) != 2:
        raise ValueError("Expect exactly two stars")

    s1, s2 = stars[0], stars[1]

    sph1 = create_sph_star(s1.mass, s1.radius, n_particles=n_sph_per_star, u_value=u_value, pos_unit=pos_unit)
    sph2 = create_sph_star(s2.mass, s2.radius, n_particles=n_sph_per_star, u_value=u_value, pos_unit=pos_unit)

    # shift to absolute positions
    sph1.position += s1.position.in_(pos_unit)
    sph2.position += s2.position.in_(pos_unit)

    sph1.velocity += s1.velocity
    sph2.velocity += s2.velocity

    gas = Particles()
    gas.add_particles(sph1)
    gas.add_particles(sph2)

    return gas

def run_fi_collision(gas, t_end=0.1 | units.yr, min_mass=1e-6 | units.MSun):
    gas = gas[gas.mass > min_mass]
    if len(gas) == 0:
        raise ValueError("All SPH particles filtered out due to low mass.")

    com_pos = gas.center_of_mass()
    com_vel = gas.center_of_mass_velocity()
    gas.position -= com_pos
    gas.velocity -= com_vel

    lengths = (gas.position).lengths()
    length_scale = lengths.max()
    if length_scale < 1e-3 | units.AU:
        length_scale = 1e-3 | units.AU

    total_mass = gas.total_mass()
    converter = nbody_system.nbody_to_si(total_mass, length_scale)
    
    hydro = Fi(converter)
    hydro.gas_particles.add_particles(gas)
    hydro.parameters.timestep = 0.001 | units.yr
    hydro.parameters.verbosity = 2

    try:
        hydro.evolve_model(t_end)
    except Exception as e:
        hydro.stop()
        raise RuntimeError("Fi crash inside evolve_model") from e

    gas_out = hydro.gas_particles.copy()
    hydro.stop()
    gas_out.position += com_pos
    gas_out.velocity += com_vel

    # Diagnostics to return instead of printing
    diagnostics = {
        "M": total_mass.in_(units.MSun),
        "Rscale": length_scale.in_(units.AU),
        "N": len(gas),
    }

    return gas_out, diagnostics


def compute_remnant_spin(gas):
    """
    Compute mass, COM velocity, spin omega, and angular momentum of remnant.
    """
    COM_pos = gas.center_of_mass()
    COM_vel = gas.center_of_mass_velocity()
    L = VectorQuantity([0.0, 0.0, 0.0], units.kg * units.m**2 / units.s)
    I_scalar = 0. | units.kg * units.m**2

    for p in gas:
        r = p.position - COM_pos
        v = p.velocity - COM_vel
        L += p.mass * r.cross(v)
        I_scalar += p.mass * r.length()**2

    omega = (L.length() / I_scalar).in_(1/units.s)
    Mbound = gas.total_mass()
    Vcom = COM_vel.in_(units.kms)
    return Mbound, Vcom, omega, L

def make_triple_binary_system(
    masses,
    seps,
    ecc,
    directions,
    centers=None,
    v_coms=None
):
    """
    Create a system of three interacting binaries with fully tunable parameters.
    """

    if not (len(masses) == 6 and len(seps) == 3 and len(directions) == 3):
        raise ValueError("Expect masses=6, seps=3, directions=3.")

    # Default centers
    if centers is None:
        centers = [
            [-300, 0, 0],
            [300, 0, 0],
            [0, 600, 0]
        ]
    # Default COM velocities
    if v_coms is None:
        v_coms = [
            [10., 0., 0.],
            [-10., 0., 0.],
            [0., -10., 0.]
        ]

    ma1, ma2, mb1, mb2, mc1, mc2 = masses
    sepA, sepB, sepC = seps
    dirA, dirB, dirC = directions
    eccA, eccB, eccC = ecc

    # Convert centers and velocities to VectorQuantity with units
    centerA = VectorQuantity(centers[0], units.AU)
    centerB = VectorQuantity(centers[1], units.AU)
    centerC = VectorQuantity(centers[2], units.AU)

    v_com_A = VectorQuantity(v_coms[0], units.kms)
    v_com_B = VectorQuantity(v_coms[1], units.kms)
    v_com_C = VectorQuantity(v_coms[2], units.kms)

    # Create binaries
    p1, p2 = make_binary(ma1, ma2, sepA | units.AU, eccA, center=centerA, direction=dirA)
    p3, p4 = make_binary(mb1, mb2, sepB | units.AU, eccB, center=centerB, direction=dirB)
    p5, p6 = make_binary(mc1, mc2, sepC | units.AU, eccC, center=centerC, direction=dirC)

    # Name particles
    p1.name, p2.name = "A1", "A2"
    p3.name, p4.name = "B1", "B2"
    p5.name, p6.name = "C1", "C2"

    # Apply COM velocities
    for p in (p1, p2):
        p.velocity += v_com_A
    for p in (p3, p4):
        p.velocity += v_com_B
    for p in (p5, p6):
        p.velocity += v_com_C

    # Combine all particles
    particles = Particles()
    for p in [p1, p2, p3, p4, p5, p6]:
        particles.add_particle(p)

    return particles



def make_binary(m1, m2, a, e=0.0, center=None, direction=0.0, orbit_plane=[0, 0, 1]):
    """
    Create a binary system with arbitrary eccentricity and orientation.

    Parameters
    ----------
    m1, m2 : float or Quantity
        Masses in Msun.
    a : Quantity
        Semi-major axis (AU).
    e : float
        Eccentricity (0=circular, 0<e<1=elliptical).
    center : VectorQuantity or list
        Center-of-mass position (default: [0,0,0] AU).
    direction : float
        Rotation angle around z-axis (radians) to orient the orbit.
    orbit_plane : list of 3 floats
        Normal vector defining orbital plane. Default: z-axis.

    Returns
    -------
    p1, p2 : Particle
        Two AMUSE particles with positions and velocities.
    """

    m1 = m1 | units.MSun
    m2 = m2 | units.MSun
    total_mass = m1 + m2

    # Default center
    if center is None:
        center = VectorQuantity([0,0,0], units.AU)
    elif not isinstance(center, VectorQuantity):
        center = VectorQuantity(center, units.AU)

    # Circular approximation if e=0
    # More generally, sample at pericenter for simplicity
    r_rel = a * (1 - e)  # separation at pericenter
    r1 = -(m2 / total_mass) * r_rel
    r2 =  (m1 / total_mass) * r_rel

    # Rotation matrix around z (or orbit_plane)
    c, s = np.cos(direction), np.sin(direction)
    R = np.array([[c, -s, 0],
                  [s,  c, 0],
                  [0,  0, 1]])

    pos1 = np.dot(R, [r1.value_in(units.AU), 0., 0.]) | units.AU
    pos2 = np.dot(R, [r2.value_in(units.AU), 0., 0.]) | units.AU

    p1 = Particle(mass=m1)
    p2 = Particle(mass=m2)
    p1.position = center + pos1
    p2.position = center + pos2

    # Circular or elliptical orbit velocity
    if e == 0.0:
        # circular orbit
        v_rel = (constants.G * total_mass / a)**0.5
    elif e < 1.0:
        # elliptical
        v_rel = ((constants.G * total_mass * float(1 + e) / (a * float(1 - e)))**0.5)
    else:
        raise ValueError("Eccentricity cannot be > or = 1")


    v1 = + (m2 / total_mass) * v_rel
    v2 = - (m1 / total_mass) * v_rel

    vel1 = np.dot(R, [0., v1.value_in(units.kms), 0.]) | units.kms
    vel2 = np.dot(R, [0., v2.value_in(units.kms), 0.]) | units.kms
    p1.velocity = vel1
    p2.velocity = vel2

    return p1, p2


def make_seba_stars(masses_msun, age):
    """
    masses_msun: list of floats (Msun)
    age: quantity with units (e.g. 3.5 | units.Myr)
    returns: seba, seba.particles (Particles with .mass, .radius, etc.)
    """
    seba = SeBa()   # fast SSE-style stellar evolution
    stars = Particles()
    for m in masses_msun:
        p = Particle(mass = m | units.MSun)
        stars.add_particle(p)
    seba.particles.add_particles(stars)
    seba.evolve_model(age)
    return seba, seba.particles

In [7]:
def run_6_body_simulation(age, masses, sep, ecc, direction, centers, v_coms, run_label=""):
    """
    Run a full 6-body simulation combining stellar dynamics, stellar evolution, and hydrodynamic mergers.

    This function sets up three binaries, evolves them under gravity, applies stellar evolution through SEBA,
    and automatically detects and resolves physical collisions between stars.When a collision is detected, 
    the function invokes an SPH (Smoothed Particle Hydrodynamics) calculation using the Fi code to simulate 
    the merger and generate a realistic remnant with updated mass, radius, and velocity. Collisions, pre/post states,
    and SPH outputs are saved to disk for later analysis.

    Parameters
    ----------
    age : float
        Initial stellar age in Myr, used for SEBA stellar evolution initialization.
    masses : list of float
        Masses (in MSun) of the six stars (two per binary).
    sep : list of float
        Initial semi-major axes (in AU) for each of the three binaries.
    ecc : list of float
        Orbital eccentricities of the binaries.
    direction : list of float
        Common orientation vector for all binaries (defines orbital plane orientation).
    centers : list of 3-element lists
        Positions (in AU) of each binary's center of mass in the simulation frame.
    v_coms : list of 3-element lists
        Center-of-mass velocities (in km/s) for each binary.
    run_label : str, optional
        Label used to name output files for this specific simulation run.

    Returns
    -------
    frames : list of Particles sets
        Snapshots of the system after each collision or major event, for visualization or replay.
    max_mass : ScalarQuantity
        Mass of the most massive star remaining at the end of the simulation (in MSun).
    max_velocity : ScalarQuantity
        Velocity magnitude of that most massive star (in km/s).

    Notes
    -----
    - Collisions are detected dynamically by comparing interstellar distances to stellar radii with a 
      configurable buffer factor.
    - SPH mergers are performed using Fi with automatic scaling of mass and size units to maintain 
      numerical stability.
    - Each collision produces pre- and post-collision snapshots, and merged remnants are reinserted 
      into the N-body system with their new properties.
    - If a collision fails or Fi crashes, the merger is skipped, and the system continues with 
      default remnant parameters.
    - The simulation terminates if all stars are ejected or merged into a single object.
    """

    # Create directories
    output_dirs = ["collisions", "final_states", "logs", "snapshots"]
    for d in output_dirs:
        if not os.path.exists(d):
            os.makedirs(d, exist_ok=True)

    target_age = age | units.Myr
    t_end = 2000 | units.yr
    dt = 5 | units.yr
    t = 0 | units.yr

    frames = []
    n_collision = 0
    last_collision_pair = None

    # Stellar evolution setup
    seba, seba_particles = make_seba_stars(masses, target_age)
    grav_particles = make_triple_binary_system(masses, sep, ecc, direction, centers, v_coms)

    total_mass = grav_particles.total_mass()
    length_scale = 1000 | units.AU
    converter = nbody_system.nbody_to_si(total_mass, length_scale)
    gravity = ph4(converter)
    gravity.particles.add_particles(grav_particles)

    for g, s in zip(gravity.particles, seba.particles):
        g.mass = s.mass
        g.radius = s.radius

    print("Starting simulation")

    # Main evolution loop
    while t < t_end:
        t += dt
        gravity.evolve_model(t)
        seba.evolve_model(target_age + t)
        frames.append(gravity.particles.copy())

        radii = [p.radius for p in gravity.particles]
        # --- Collision detection block ---
        pair = detect_close_pair(gravity.particles, radii)

        if pair:
            i, j, sep = pair
            print(f"Collision detected at {t.value_in(units.yr):.1f} yr between indices {i} and {j}")

            # Take snapshots
            pre_snapshot = gravity.particles.copy()
            particle_keys = {p.key: idx for idx, p in enumerate(gravity.particles)}
            pre_collision_filename = os.path.join(
                "collisions", f"pre_collision_{n_collision}_{run_label}.amuse"
            )
            write_set_to_file(pre_snapshot, pre_collision_filename, "amuse", overwrite_file=True)

            # Grab colliding particles
            p_i = gravity.particles[i]
            p_j = gravity.particles[j]

            # --- Generate SPH particles ---
            colliders_for_sph = Particles()
            colliders_for_sph.add_particle(p_i.copy())
            colliders_for_sph.add_particle(p_j.copy())
            sph = make_sph_from_two_stars(colliders_for_sph, n_sph_per_star=50)

            # Filter out bad particles
            sph = sph[sph.mass > 0 | units.MSun]
            if len(sph) == 0:
                print(" SPH particles all filtered out, skipping collision")
                continue

            # Center SPH on COM
            com_pos = sph.center_of_mass()
            com_vel = sph.center_of_mass_velocity()
            sph.position -= com_pos
            sph.velocity -= com_vel

            # Save SPH input
            write_set_to_file(
                sph,
                os.path.join("collisions", f"collision_{n_collision}_sph_input_{run_label}.amuse"),
                "amuse",
                overwrite_file=True,
            )

            # --- Run Fi ---
            try:
                gas_out, diag = run_fi_collision(sph, t_end=0.1 | units.yr)
                print("Fi collision done:", diag)
            except Exception as e:
                print(f"Fi failed for collision {n_collision}: {e}")
                continue

            
            write_set_to_file(
                gas_out,
                os.path.join("collisions", f"collision_{n_collision}_sph_output_{run_label}.amuse"),
                "amuse",
                overwrite_file=True,
            )


            # --- Compute remnant ---
            try:
                Mbound, Vcom, omega, L = compute_remnant_spin(gas_out)
                COM_pos = gas_out.center_of_mass()
                COM_vel = gas_out.center_of_mass_velocity()
                # Transform back to global coordinates
                COM_pos += (p_i.position + p_j.position) / 2
                COM_vel += (p_i.velocity + p_j.velocity) / 2


                rs = np.sqrt((gas_out.x - COM_pos[0])**2 +
                            (gas_out.y - COM_pos[1])**2 +
                            (gas_out.z - COM_pos[2])**2)
                r90 = np.percentile([rv.value_in(units.RSun) for rv in rs], 90)
                remnant_radius = r90 | units.RSun

            except Exception:
                print(f"Failed to compute remnant properties, using defaults.")
                Mbound = p_i.mass + p_j.mass
                COM_pos = (p_i.position + p_j.position) / 2
                COM_vel = (p_i.velocity + p_j.velocity) / 2
                omega = 0 | units.rad/units.s
                remnant_radius = (Mbound.value_in(units.MSun)**0.57) | units.RSun

            # Assign remnant to particle i
            p_i.mass = Mbound
            p_i.position = COM_pos
            p_i.velocity = COM_vel
            p_i.radius = remnant_radius

            # Remove merged particle
            gravity.particles.remove_particle(p_j)
            gravity.recommit_particles()

            # Save post-collision snapshot
            post_snapshot = gravity.particles.copy()
            frames.append(post_snapshot)
            n_collision += 1

            print(f"Collision {n_collision} processed: remnant mass = {Mbound.value_in(units.MSun):.2f} MSun")


    # BEFORE stopping gravity
    final_particles = gravity.particles.copy() 

    gravity.stop()
    seba.stop()

    if len(final_particles) == 0:
        print(" No particles remaining in the system! Returning defaults.")
        max_mass_particle = None
        max_mass = 0 | units.MSun
        max_velocity = 0 | units.kms
    else:
        max_mass_particle = max(final_particles, key=lambda p: p.mass)
        max_mass = max_mass_particle.mass
        max_velocity = max_mass_particle.velocity.length()

        print(f"Most massive star: Mass = {max_mass.value_in(units.MSun):.2f} MSun, "
              f"Velocity = {max_velocity.value_in(units.kms):.2f} km/s")


    # Save final system
    final_filename = os.path.join("final_states", f"final_system_{run_label}.amuse")
    write_set_to_file(final_particles, final_filename, "amuse", overwrite_file=True)

    return frames, max_mass, max_velocity

# Examples of 6 body evolution (3 binaries)

## No encounter

In [8]:
# Initial conditions
age = 3.5 #Myr
masses = [90, 10,  20, 70,  10, 10] #Msun
sep= [30, 20, 50] #AU
ecc = [0.0, 0.0, 0]
centers = [[-100, 0, 0], [300, 0, 0], [0, 600, 0]] #AU
v_coms = [[30., 3., 0.], [-10., -21., 0.], [5., -20., 1.] ] # km/s
direction = [0.4, -0.6, 1.2]


no_colission , _, _= run_6_body_simulation(age, masses,sep,ecc, direction, centers, v_coms, "Test_No_Collision")

Starting simulation
Most massive star: Mass = 65.88 MSun, Velocity = 33.75 km/s


In [9]:
visualize_frames(no_colission, "Test_No collision")

 GIF saved as Gif-6body/encounter_evolution_Test_No collision.gif


## Encounter

In [10]:
# Initial conditions
age = 3.5  # Myr
masses = [90, 10,  20, 20,  40, 10]  # Msun
sep = [10, 80, 10]  #AU
ecc = [0.5, 0.0, 0.0]  
centers = [[-20, 0, 0], [20, 100, 0], [200, 0, 0]] #AU
v_coms = [[0.3, 0.1, 0.4], [0.1, -1, -2], [0.5, 0, -0.1]] # km/s
direction = [0, 0, 0.1] 

#Run
collision, _, _ = run_6_body_simulation(age, masses, sep, ecc, direction, centers, v_coms, "Test_Collision")


Starting simulation
Collision detected at 275.0 yr between indices 0 and 4
Fi collision done: {'M': quantity<104.741278532 MSun>, 'Rscale': quantity<15.6969690064 au>, 'N': 100}
Collision 1 processed: remnant mass = 104.74 MSun
Collision detected at 280.0 yr between indices 0 and 2
Fi collision done: {'M': quantity<124.696550674 MSun>, 'Rscale': quantity<456.17901637 au>, 'N': 100}
Collision 2 processed: remnant mass = 124.70 MSun
Most massive star: Mass = 124.70 MSun, Velocity = 25.00 km/s


In [ ]:
visualize_frames(collision, "Collision")